# Schrödinger equation

Run setup cell; then pick a scheme and run the second cell to interact with the simulation.

### Setup

In [1]:
from __future__ import print_function
%matplotlib inline

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### Pick a scheme and a test
Slide time slider for plot to appear. Pick scheme to reset. Resolution of $1$ corresponds to standard settings. All changes relative.
Not all schemes and parameter combinations are stable. 

**Slide time slider carefully**. 

In [3]:
%run src/sps_test_suite.py

def pick_scheme(scheme_name, test_name, resolution, eta):
    global solver, wsolver 
    c = getBaseConfig()
    test, testConfig, potential = test_list[nice_test_list[test_name]]
    scheme, schemeConfig = scheme_list[nice_scheme_list[scheme_name]]
    schemeConfig(c)
    testConfig(c)

    c["resolution"] = int(c["resolution"] * resolution)
    c["dpi"] = 200
    c["figsize"] = [3.54, 2.54]
    c["plotDebug"] = False
    c["hbar"] = eta 
    c["m"] = 1

    solver = scheme(c, test)
    solver.setExternalPotentialFunction(potential)
    wsolver = wave_schemes.SpectralScheme(c, test)
    wsolver.setExternalPotentialFunction(potential)

a = widgets.Dropdown(description="pick scheme",  value = "hybrid scheme", options = nice_scheme_list.keys())
b = widgets.Dropdown(description="pick test", value = "cosmo with gravity (2D)", options = nice_test_list.keys())
c = widgets.FloatSlider(description="resolution", min=.1, max=4, step=.1, value=1)
d = widgets.FloatSlider(description="eta", min=.25, max=4, step=.05, value=1)
ui = widgets.HBox([a, b, c, d])

scheme_picker = widgets.interactive_output(pick_scheme, {'scheme_name': a, 'test_name': b, 'resolution': c, 'eta' : d})

display(ui, scheme_picker)

def run(t):
    global solver, wsolver 

    solver.tEnd = solver.config["tEnd"] * t/100
    wsolver.tEnd = solver.tEnd
    solver.config["savePlots"] = False

    if solver.dimension == 1:
        solver.config["dpi"] = 100
    else:
        solver.config["dpi"] = 200

    if solver.dimension == 1:
        solver.run()
        animation.drawFrame(solver)
    else:
        solver.config["plotTime"] = False 
        solver.run()
        wsolver.run()
        animation.drawFrame(solver, waveSolver=wsolver)

    plt.show()


interactive_plot= interactive(run,
t=widgets.FloatSlider(min=0, max=200, step=1, value=0),
)


output = interactive_plot.children[-1]
output.layout.height = '500px'
buttons = widgets.HBox(interactive_plot.children[:-1])
display(output)
display(buttons)


Output()

Output(layout=Layout(height='500px'))